# Verificação Indutiva

Na aula passada vimos como modelar sistemas dinâmicos com *First-order Transition Systems* (FOTSs) e como verificar propriedades de segurança e animação sobre estes sistemas com *Bounded Model Checking* (BMC). O procedimento de BMC verifica se uma propriedade é válida num FOTS considerando apenas um número limitado de estados de execução. Se quisermos verificar uma propriedade para qualquer execução não limitada do FOTS temos que usar um procedimento alternativo. Nesta aula vamos usar indução para fazer verificação em traços não limitadados.

## Verificação indutiva de invariantes

No caso da verificação de propriedades de segurança $G\ \phi$, para verificar o invariante $\phi$ por indução temos que verificar as seguintes condições:
- $\phi$ é válido nos estados iniciais, ou seja, $\mathit{init}(s) \rightarrow \phi(s)$
- Para qualquer estado, assumindo que $\phi$ é verdade, se executarmos uma transição, $\phi$ continua a ser verdade no próximo estado, ou seja, $\phi(s) \wedge \mathit{trans}(s,s') \rightarrow \phi(s')$.

 $$ \neg(init(s)\rightarrow inv(s))$$
 $$\neg(\neg init(s)\rightarrow inv(s))$$
 $$init(s)\wedge \neg inv(s)$$

### Exercício 1

Implemente o método `induction_always` para verificação de invariantes por indução. Note que terá que testar a validade das duas condições acima recorrendo à satisfiabilidade, ou seja, usando o solver para encontrar contra-exemplos, devendo o procedimento reportar qual das propriedades falha. Por exemplo, no caso da primeira deve procurar uma valoração que satisfaça $\mathit{init}(s) \wedge \neg \phi(s)$.

In [7]:
from z3 import *

def induction_always(declare,init,trans,inv):
    # completar
    state=declare(0)
    s=Solver()
    s.add(init(state),Not(inv(state)))
    if s.check()==sat:
        m=s.model()
        print("O invariante é falso no seguinte estado inicial:")
        for v in state:
            print(v,'=',m[state[v]])
        return
    assert(s.check()==unsat)
    s=Solver()
    state0=declare(0)
    state1=declare(1)
    s.add(inv(state0))
    s.add(trans(state0,state1))
    s.add(Not(inv(state1)))
    if s.check()==sat:
          m=s.model()
          print("O passo indutivo falhou no seguinte estado:")
          for v in state0:
              print(v,'=',m[state0[v]])
          return
    assert(s.check()==unsat)
    print("Propriedade valida!")


Usando este procedimento podemos tentar verificar a propriedade $G (x \ge 0)$ sobre o programa

```Python
{ x >= 3 }
0: while (x>0):
1:    x = x-1
2: stop
```

In [5]:
def declare(i):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = Int('x'+str(i))
    return state

def init(state):
    return And(state['pc'] == 0, state['x']>=3)

def trans(curr,prox):
    t01 = And(curr['pc'] == 0,curr['x'] > 0,prox['pc'] == 1,prox['x'] == curr['x'])
    t02 = And(curr['pc'] == 0,curr['x'] <= 0,prox['pc'] == 2,prox['x'] == curr['x'])
    t10 = And(curr['pc'] == 1,prox['pc'] == 0,prox['x'] == curr['x']-1)
    t22 = And(curr['pc'] == 2,prox['pc'] == 2,prox['x'] == curr['x'])
    return Or(t02,t01,t10,t22)

def positive(state):
    return (state['x'] >= 0)

induction_always(declare,init,trans,positive)

o passo indutivo falhou no seguinte estado:
pc = 1
x = 0


Como se pode verificar, a propriedade anterior, embora seja verdade, não pode ser verificada por indução, pois o passo indutivo falha quando $\mathit{pc} = 1$ e $x = 0$. De facto, este contra-exemplo para o passo de indução não é realista, dado que este estado não é acessível a partir do estado inicial. Para verificarmos esta propriedade por indução temos que encontrar um invariante $\phi$ que seja mais forte que o desejado, ou seja, $\phi(x,\mathit{pc}) \rightarrow x \ge 0$, e que seja verificável por indução.

### Exercício 2

Encontre uma propriedade mais forte que $x \ge 0$ e que seja verificável por indução.

$(pc = 1 \wedge x >0) \vee (pc \not= 1 \wedge x \geq 0)$

In [6]:
# completar
def positive_stronger(state):
    return (If(state['pc']==1,state['x']>0,state['x'] >=0))

induction_always(declare,init,trans,positive_stronger)

propriedade valida


### Exercício 3

Utilize o Z3 para verificar que essa propriedade é realmente mais forte que $x \ge 0$.

In [8]:
# completar
s=Solver()
state= declare(0)
s.add(positive_stronger(state))
s.add(Not(positive(state)))
if s.check() == unsat:
    print("positive_stronger é realmente mais forte que positive")

positive_stronger é realmente mais forte que positive


Nem sempre é fácil encontrar um invariante indutivo mais forte que o desejado. Um procedimento de verificação alternativo que pode ser utilizado nesses casos é a designada $k$-indução. A ideia é generalizar a indução simples assumindo no passo indutivo que o invariante é válido nos $k$ estados anteriores. Claro que neste caso também será necessário verificar que a propriedade é válida nos $k$ primeiros estados. Com esta generalização é possível verificar que $x \ge 0$ é um invariante recorrendo a $k$-indução com $k=2$.

### Exercício 4

Implemente o método `kinduction_always` para verificação de invariantes por $k$-indução.

In [17]:
def kinduction_always(declare,init,trans,inv,k):
    # completar
    s=Solver()
    state=[declare(i) for i in range(k)]
    s.add(init(state[0]))
    s.add(And([trans(state[i],state[i+1]) for i in range(k-1)]))
    s.add(Or([Not(inv(state[i])) for i in range(k)]))
    if s.check()==sat:
        print("O invariante é falso num dos primeiros",k,"estados")
        return
    assert(s.check()==unsat)
    s=Solver()
    state=[declare(i) for i in range(k+1)]
    s.add(init(state[0]))
    s.add(And([trans(state[i],state[i+1]) for i in range(k)]))
    s.add(And([inv(state[i]) for i in range(k)]))
    s.add(Not(inv(state[k])))
    if s.check()==sat:
        print("Não é possivel provar o invariante com",k,"indução")
        return
    assert(s.check()==unsat)
    print("Propriedade valida!")

    
kinduction_always(declare,init,trans,positive,2)

Propriedade valida!


## Verificação de propriedades de animação por indução

Como vimos, podemos verificar propriedades de animação do tipo $F\ \phi$ usando BMC. Mais uma vez, se quisermos verificar estas proprieades para qualquer execução ilimitada temos que usar um procedimento alternativo. Uma possibilidade consiste em reduzir a verificação dessas propriedades à verificação de uma propriedade de segurança, mais concretamente um invariante, que possa ser verificado por indução.

Por exemplo, uma técnica que pode ser usada para verificar uma propriedade de animação do tipo $F\ (G\ \phi)$, passa por descobrir um *variante* $V$ que satisfaz as seguintes condições:
- O variante é sempre positivo, ou seja, $G\ (V(s) \ge 0)$
- O variante descresce sempre (estritamente) ou atinge o valor 0, ou seja, $G\ (\forall s' . \mathit{trans}(s,s') \rightarrow (V(s') < V(s) \vee V(s') = 0))$
- Quando o variante é 0 verifica-se necessariamente $\phi$, ou seja, $G\ (V(s)=0 \rightarrow \phi(s))$

A terminação de um programa é um dos exemplos de propriedade que encaixa neste padrão. Por exemplo, para o programa acima a terminação pode ser expressa por $F\ (G\ \mathit{pc}=2)$ (note que devido ao lacete no estado final, o programa permanece indefinidamente no estado final).

Note que a segunda condição recorre a uma quantificação universal sobre os possíveis estados alcançáveis por uma transição a partir de cada estado.

### Exercício 5

Descubra o variante que permite provar por indução que o programa acima termina. Encontre também os menores valores de $k$ que permitem provar as condições acima por $k$-indução.

In [31]:
# completar
def variante(state):
    return 2 + 2*state['x'] - state['pc']

def termina(state):
    return (state['pc'] == 2)

def variante_positivo(state):
    return variante(state)>=0

kinduction_always(declare,init,trans,variante_positivo,3)

def variante_decrescente(state):
    nextstate=declare(-1)
    return ForAll(list(nextstate.values()),Implies(trans(state,nextstate),Or(variante(nextstate)< variante(state),variante(nextstate)==0)))

kinduction_always(declare,init,trans,variante_decrescente,3)

def variante_util(state):
    return Implies(variante(state)==0,termina(state))

kinduction_always(declare,init,trans,variante_util,4)

Propriedade valida!
Propriedade valida!
Propriedade valida!


### Exercício 6

Encontrar um variante que decresça em todas as transições nem sempre é fácil. Podemos relaxar a segunda condição acima e exigir que o variante apenas tenha que decrescer estritamente a cada $l$ transições. Neste caso dizemos que temos um *lookahead* de $l$. Altere a definição da função decrescente por forma a considerar um lookahead de 2 e encontre um variante mais simples que o anterior que lhe permita verificar a terminação do programa.

In [ ]:
# completar
def variante(state):
    return if(state['pc']==2,0,1+state['x'])